In [1]:
# import 
import json
import sys

import pandas

In [2]:
# setting

## columns
COLUMNS_CALCED = [
    'average_all', 'average', 'average_exclude_debut', 'highest', 'highest_exclude_debut',
    'over30', 'over50', 'over100', 'over200', 'count_all', 'count' #, 'under30', 'under50', 'under100', 'under200', 
]
COLUMNS = [
    'title', 'date', 'initial', 'kind', 'r2019u',
    'r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019',
] + COLUMNS_CALCED
RECENT = 'r2019'
BEFORE = 'r2018'
URA = 'r2019'
RANK_INCOMPLETE = ['r2005', 'r2006#1', 'r2006#2']
RANK_COMPLETE = ['r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019']
RANK_ALL = RANK_INCOMPLETE + RANK_COMPLETE
RANK_ALL_LAST_YEAR = RANK_INCOMPLETE + RANK_COMPLETE[:-1]
RANK = RANK_ALL[3:]
RANK_LAST_YEAR = RANK_ALL_LAST_YEAR[3:]
RANK_BEFORE = list(filter(lambda x: x != RECENT, RANK_ALL))

def year_filter(value: str, initial: str, array: list, mode: str = 'ge'):
    if mode not in ['gt', 'ge']:
        raise ValueError(f'invalid mode: {mode}')
    if value in array and initial in array:
        if mode == 'ge':
            return array.index(value) >= array.index(initial)
        else:
            return array.index(value) > array.index(initial)
    elif value in array:
        return True
    else:
        return False

### summary.json

In [3]:
# read json
SUMMARY_PATH = '../data/summary.json'
with open(SUMMARY_PATH, encoding='utf8') as summary_file:
    summary = json.load(summary_file)

In [4]:
# make main dataframe
df = pandas.DataFrame(summary)
df.index = df.no
df['average_all'] = df.average
df = df.drop(columns=['no', 'average'], axis=0)
df.date = pandas.to_datetime(df.date)
df.head()

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019,average_all
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,96.0,123.0,68.0,169.0,165.0,100.0,134.0,164.0,123.0,121.462
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,72.0,57.0,54.0,87.0,81.0,59.0,146.0,126.0,54.0,80.923
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,175.0,141.0,218.0,141.0,238.0,190.0,155.0,164.0,216.0,171.769
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,204.0,131.0,174.0,194.0,253.0,244.0,202.0,250.0,260.0,201.077
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,81.0,96.0,150.0,106.0,156.0,244.0,187.0,63.0,106.0,125.615


### works.json

In [5]:
WORKS_PATH = '../data/works.json'
with open(WORKS_PATH, encoding='utf8') as works_file:
    works = json.load(works_file)

In [6]:
dfw = pandas.DataFrame(works)
dfw.date = pandas.to_datetime(dfw.date)
dfw.head()

,title,kind,date
0,SOME GIRLS -REBEL STREEL IV,OT,1997-04-25
1,幻想庭園,OA,1988-01-25
2,フラワーチャイルド,S,1988-07-25
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07
4,月下の一群,OA,1992-12-09


### info.json

In [7]:
INFO_PATH = '../json/info.json'
with open(INFO_PATH, encoding='utf8') as info_file:
    info = json.load(info_file)

lowest = info['max']

In [8]:
df = df.assign(**{key: 0. for key in ['average', 'average_exclude_debut', 'highest', 'highest_exclude_debut', 'lowest']}) \
       .assign(**{key: 0 for key in ['over30', 'over50', 'over100', 'over200', 'count_all', 'count']}) \
       .assign(latest_work=df.works.apply(lambda l: l[-1]))

for no in df.index:
    song = df.loc[no]
    to_mean = list(filter(lambda y: year_filter(y, song.initial, RANK), RANK))
    to_mean_exclude_debut = list(filter(lambda y: year_filter(y, song.initial, RANK, 'gt'), RANK))
    df.average.at[no] = song[to_mean].mean()
    df.average_exclude_debut.at[no] = song[to_mean_exclude_debut].mean()
    df.highest_exclude_debut.at[no] = song[to_mean_exclude_debut].min()
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    df.highest.at[no] = song[to_mean_all].min()
    df.lowest.at[no] = song[to_mean_all].max()
    df.over30.at[no] = len(list(filter(lambda s: s > 30, song[to_mean_all].values)))
    df.over50.at[no] = len(list(filter(lambda s: s > 50, song[to_mean_all].values)))
    df.over100.at[no] = len(list(filter(lambda s: s > 100, song[to_mean_all].values)))
    df.over200.at[no] = len(list(filter(lambda s: s > 200, song[to_mean_all].values)))
    df.count_all.at[no] = len(to_mean_all)
    df.at[no, 'count'] = len(to_mean)
    # latest work


df.head()
# df.iloc[145]

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,highest,highest_exclude_debut,lowest,over30,over50,over100,over200,count_all,count,latest_work
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,68.0,68.0,169.0,13,13,9,0,16,13,月光嗜好症
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,54.0,54.0,146.0,13,13,2,0,16,13,étoiles
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,141.0,141.0,238.0,13,13,13,3,16,13,神々の黄昏
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,131.0,131.0,260.0,13,13,13,6,16,13,幻想庭園
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,63.0,63.0,244.0,13,13,9,1,16,13,神々の黄昏


## URA Ranking

In [9]:
# ura ranking
ura = df[df[URA] > 100][COLUMNS]
ura['rank_average'] = ura.average.rank(method='min', na_option='bottom')
ura['rank_average_exclude_debut'] = ura.average_exclude_debut.rank(method='min', na_option='bottom')
ura['rank_highest'] = ura.highest.rank(method='min', na_option='bottom')
ura['rank_highest_exclude_debut'] = ura.highest_exclude_debut.rank(method='min', na_option='bottom')

ura.head()

,title,date,initial,kind,r2019u,r2007,r2008,r2009,r2010,r2011,...,over30,over50,over100,over200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,r2005,"[SA, OA, OT]",64.0,86.0,126.0,116.0,109.0,96.0,...,13,13,9,0,16,13,58.0,46.0,97.0,64.0
3,マリーゴールド・ガーデン,1988-01-25,r2005,"[SA, OA, S-2]",128.0,141.0,162.0,142.0,150.0,175.0,...,13,13,13,3,16,13,126.0,120.0,183.0,164.0
4,鏡面界 im Juni,1988-01-25,r2005,[OA],150.0,165.0,185.0,181.0,171.0,204.0,...,13,13,13,6,16,13,176.0,162.0,169.0,152.0
5,アンジェ・ノワールの祭戯,1988-01-25,r2005,"[SA, OA]",NaN,109.0,131.0,111.0,93.0,81.0,...,13,13,9,1,16,13,65.0,52.0,87.0,55.0
6,紅い睡蓮の午後,1988-01-25,r2005,"[SA, OA]",152.0,144.0,176.0,153.0,164.0,188.0,...,13,13,13,5,16,13,150.0,143.0,178.0,159.0


In [10]:
col = 4
length = len(ura)
row = int(pandas.np.ceil(length/col))

a = [[] for _ in range(col)]
c = 0
for no in ura.index:
    a[int(c // row)] += [ura.title.at[no]]
    c += 1

for i in range(row):
    print('|'.join([a[j][i] if j * row + (i + 1) <= length else '' for j in range(col)]))

桜の花は狂い咲き|病める薔薇|眠れる豹|真偽贋物遊覧会
マリーゴールド・ガーデン|MALICE|血の断章|La vérité
鏡面界 im Juni|à la cuisine|薨|野性双生児
アンジェ・ノワールの祭戯|桂花葬|小さき者への贖罪のためのアリア|逝ける王女の肖像画
紅い睡蓮の午後|閉ざされた画室|鬼帝の剣|真夏の憂愁夫人
少女忌恋歌|un tableau blanc ～絵画旅行～|桂冠詩人|天譴と超克
パラソルのある風景|闇の翼ですべてをつつむ夜のためのアリア|裸々イヴ新世紀|快恠奇奇
硝子天井のうちゅう|après le noir|騎士乙女|Fräulein Rose
幻想庭園|宵待草|吾君想う故に吾在り生き霊となりて|乙女の贖い
フラワーチャイルド|La vie en rose|戦慄の子供たち|令嬢薔薇図鑑
恋せよ乙女 -Love story of ZIPANG-|schism|Hell's Maria|ローズ家の双子達
ビアンカ|時の森のソワレ|処女懐胎、あるいは白骨塔より少女達は飛翔する|隼の白バラ
空宙舞踏会|夢魔の夜、あなたを迎える者がある|お毒味LADY|薔薇娼館
木洩れ陽のワルツ|プラトニック|Animals on the Earth|朗読する女中と小さな令嬢
堕ちて候|Féliciter|上海繚乱ロマンチカ|黒百合隠密カゲキダン
月光浴|MOTHER|世紀末ゲネシス|南無地獄大菩薩
楽園喪失 -Paradise Lost-|繭|この國の向こうに|見ぬ友へ
舞踏会の手帖|Je te veux|輪廻闇妖散華|PANDEMIC
エスカルゴ嬉遊曲|Après un rêve|堕天國宣戦|赤い蝋燭と金魚
サロメティック・ルナティック|Daphné|寶島|Femme Fatale
雪のひとひら|わが恋は月光に散りぬ|月光オペラの天鵞絨の緞帳は上がり|海賊皇女
ダリの宝石店|pastel pure|恋の翼は真珠色の骨を持ち|静謐なる私小説
Virtual Fantasy|あたしがアリスだった頃|私の心は語る薔薇の御胸を葩の御身を|形而上的な、蝶になる
小夜啼鳥|君がため、惜しからざりし命さへ|エルフの娘は地上に降り|極楽鳥歌
星降る夜の天文学 ～BEDSIDE ASTRONOMY～|肉体の悪魔|百合の日々は追憶の中に潜み薫る|遊蝶花ワルツ
オフェリア

### song list

In [11]:
length = len(ura)
row = int(pandas.np.ceil(length/2))

a2 = [[] for _ in range(col)]
c = 0
for no in ura.index:
    a2[int(c // row)] += [ura.title.at[no]]
    c += 1

# write to file (html tag)
with open('../data_ura/list.txt', 'w', encoding='utf8') as list_file:
    list_file.write('<div style="display: flex; justify-content: space-around; border: solid 1px; overflow: auto; flex-wrap: wrap;">\n')

    for i in range(2):
        j = 0
        list_file.write(f'<ol start="{i * row + j + 1}" style="padding-left: 5px;">\n')
        for j in range(len(a2[i])):
            list_file.write(f'<li>{a2[i][j]}</li>')
        list_file.write('\n</ol>\n')
    list_file.write('</div>')

### last year

In [12]:
last = df.query(f'{BEFORE} >= 100')

for no in last.index:
    song = last.loc[no]
    to_mean = list(filter(lambda y: year_filter(y, song.initial, RANK_LAST_YEAR), RANK_LAST_YEAR))
    to_mean_exclude_debut = list(filter(lambda y: year_filter(y, song.initial, RANK_LAST_YEAR, 'gt'), RANK_LAST_YEAR))
    last.average.at[no] = song[to_mean].mean()
    last.average_exclude_debut.at[no] = song[to_mean_exclude_debut].mean()
    last.highest_exclude_debut.at[no] = song[to_mean_exclude_debut].min()
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL_LAST_YEAR), RANK_ALL_LAST_YEAR))
    last.highest.at[no] = song[to_mean_all].min()
    last.lowest.at[no] = song[to_mean_all].max()
    last.over30.at[no] = len(list(filter(lambda s: s > 30, song[to_mean_all].values)))
    last.over50.at[no] = len(list(filter(lambda s: s > 50, song[to_mean_all].values)))
    last.over100.at[no] = len(list(filter(lambda s: s > 100, song[to_mean_all].values)))
    last.over200.at[no] = len(list(filter(lambda s: s > 200, song[to_mean_all].values)))
    last.count_all.at[no] = len(to_mean_all)
    last.at[no, 'count'] = len(to_mean)

last['under100'] = (last.count_all - last.over100) / last.count_all
last['rank_average'] = last.average_exclude_debut.rank(method='min', na_option='bottom')
last['rank_highest'] = last.highest_exclude_debut.rank(method='min', na_option='bottom')

In [32]:
tmp = last.sort_values(['r2019u']).query('r2019u <= 20')[['title','date','r2019u', RECENT, 'under100', 'rank_average', 'average_exclude_debut', 'rank_highest', 'highest_exclude_debut']]

with open('../data_ura/review.md', 'w', encoding='utf8') as file:
    file.write('ウラ順位|曲名|発表年|2019本戦|100位圏内率|平均順位の順位|最高順位の順位\n')
    file.write('---:|---|---:|---:|---:|---:|---:\n')
    for no in tmp.index:
        row = tmp.loc[no]
        file.write(f'''{int(row.r2019u)}|{'<span style="color: red;">' if int(row.r2019) <= 100 else ''}{row.title}{'</span>' if int(row.r2019) <= 100 else ""}|{row.date.year}|{int(row.r2019)}|{row.under100:.3f}|{int(row.rank_average)}|{int(row.rank_highest)}\n''')

tmp

,title,date,r2019u,r2019,under100,rank_average,average_exclude_debut,rank_highest,highest_exclude_debut
no,,,,,,,,,
131,密猟区,2005-06-22,1.0,96.0,0.733333,13.0,82.250000,17.0,34.0
95,遊月恋歌,2002-07-24,2.0,49.0,0.933333,1.0,47.083333,4.0,18.0
207,阿芙蓉寝台,2009-08-26,3.0,71.0,0.888889,2.0,58.000000,6.0,24.0
111,少女殉血,2003-12-03,4.0,71.0,0.533333,44.0,114.750000,10.0,27.0
97,Lolita in the garret,2002-07-24,5.0,56.0,0.866667,3.0,64.916667,21.0,36.0
198,裸々イヴ新世紀,2009-01-21,6.0,141.0,0.700000,21.0,96.666667,12.0,29.0
77,少女貴族,2001-04-25,7.0,58.0,0.800000,9.0,80.500000,6.0,24.0
88,妄想水族館,2001-07-25,8.0,92.0,0.800000,6.0,75.083333,1.0,14.0
18,マダム・ノワール -Madame Noir,1992-12-09,9.0,69.0,0.533333,22.0,98.000000,8.0,25.0


### ura in last year

In [14]:
tmp = ura.sort_values(['r2019u']).query('r2019u <= 30')[['title', 'r2019u', 'r2019', 'date']]

with open('../data_ura/top_in_last_ura.md', 'w', encoding='utf8') as file:
    file.write('ウラ順位|曲名|発表年|2019本戦\n')
    file.write('---:|---|---:|---:\n')
    for no in tmp.index:
        row = tmp.loc[no]
        file.write(f'{int(row.r2019u)}|{row.title}|{row.date.year}|{int(row.r2019)}\n')

tmp

,title,r2019u,r2019,date
no,,,,
198,裸々イヴ新世紀,6.0,141.0,2009-01-21
186,黙示録前戯,10.0,125.0,2008-08-27
215,堕天國宣戦,11.0,122.0,2009-10-21
240,Troubadour,13.0,111.0,2011-05-25
271,薔薇娼館,17.0,117.0,2013-09-11
187,ヘテロ失楽園,18.0,101.0,2008-08-27
235,鹿鳴館ブギウギ,19.0,137.0,2010-09-26
281,PANDEMIC,20.0,110.0,2014-08-27
138,百合と夜鶯,21.0,192.0,2005-12-07


### top 100

In [15]:
top100 = ura.loc[:, :]
top100['under100'] = top100.count_all - top100.over100
top100['under100_rate'] = top100.under100 / top100.count_all
top100['rank_under100_rate'] = top100.under100_rate.rank(method='min', na_option='bottom', ascending=False)
tmp = top100.sort_values(['under100_rate', 'under100'], ascending=False) \
   .query('under100_rate > 0.7') \
   [['title'] + RANK_COMPLETE + ['under100',  'under100_rate', 'rank_under100_rate', 'date']]

with open('../data_ura/top100.txt', 'w', encoding='utf8') as file:
   file.write('<table>\n')
   file.write('<tr><th>順位</th><th>曲名</th><th>発表年</th><th>回数</th><th>率</th></tr>\n')
   before = 0
   for no in tmp.index:
      row = tmp.loc[no]
      file.write('<tr>')
      if before != row.rank_under100_rate:
         count = sum(tmp.rank_under100_rate == row.rank_under100_rate)
         file.write(f'''<td{'' if count < 2 else f' rowspan="{count}"'}>{int(row.rank_under100_rate)}</td>''')
      file.write(f'<td>{row.title}</td><td>{row.date.year}</td><td>{row.under100}</td><td>{row.under100_rate:.3f}</td></tr>\n')
      before = row.rank_under100_rate
   file.write('</table>')

tmp

,title,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019,under100,under100_rate,rank_under100_rate,date
no,,,,,,,,,,,,,,,,,,
130,昭和恋々幻燈館,37.0,41.0,63.0,67.0,56.0,52.0,75.0,95.0,51.0,69.0,41.0,51.0,117.0,15,0.937500,1.0,2005-06-22
181,コトダマ,NaN,8.0,32.0,35.0,38.0,53.0,51.0,62.0,44.0,56.0,72.0,83.0,211.0,11,0.916667,2.0,2008-01-23
185,神風,NaN,NaN,13.0,21.0,14.0,24.0,16.0,27.0,39.0,44.0,24.0,40.0,145.0,10,0.909091,3.0,2008-08-27
261,真夏の憂愁夫人,NaN,NaN,NaN,NaN,NaN,39.0,56.0,29.0,9.0,33.0,37.0,49.0,101.0,7,0.875000,4.0,2012-07-18
264,快恠奇奇,NaN,NaN,NaN,NaN,NaN,NaN,13.0,33.0,53.0,42.0,68.0,52.0,151.0,6,0.857143,5.0,2013-02-20
268,令嬢薔薇図鑑,NaN,NaN,NaN,NaN,NaN,NaN,6.0,35.0,57.0,77.0,25.0,44.0,117.0,6,0.857143,5.0,2013-09-11
269,ローズ家の双子達,NaN,NaN,NaN,NaN,NaN,NaN,3.0,12.0,47.0,45.0,26.0,78.0,106.0,6,0.857143,5.0,2013-09-11
195,鬼帝の剣,NaN,NaN,5.0,32.0,30.0,56.0,72.0,56.0,102.0,90.0,57.0,23.0,125.0,9,0.818182,8.0,2008-11-19
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,NaN,NaN,NaN,17.0,31.0,19.0,34.0,26.0,71.0,109.0,58.0,70.0,151.0,8,0.800000,9.0,2009-08-26


### first over100

In [36]:
tmp = ura[(100 < ura[RECENT])].query('over100 == 1')[['title', 'date', *RANK_COMPLETE]]

with open('../data_ura/first100.md', 'w', encoding='utf8') as file:
    file.write('曲名|発表年|最低順位(2019 以前)|2019\n')
    file.write('---|---:|---:|---:\n')
    for no in tmp.index:
        row = tmp.loc[no]
        file.write(f'{row.title}|{row.date.year}|{int(row[RANK_COMPLETE[:-1]].max()) if row.date.year < 2019 else "-"}|{int(row.r2019)}\n')

tmp

,title,date,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,
130,昭和恋々幻燈館,2005-06-22,37.0,41.0,63.0,67.0,56.0,52.0,75.0,95.0,51.0,69.0,41.0,51.0,117.0
181,コトダマ,2008-01-23,NaN,8.0,32.0,35.0,38.0,53.0,51.0,62.0,44.0,56.0,72.0,83.0,211.0
185,神風,2008-08-27,NaN,NaN,13.0,21.0,14.0,24.0,16.0,27.0,39.0,44.0,24.0,40.0,145.0
261,真夏の憂愁夫人,2012-07-18,NaN,NaN,NaN,NaN,NaN,39.0,56.0,29.0,9.0,33.0,37.0,49.0,101.0
264,快恠奇奇,2013-02-20,NaN,NaN,NaN,NaN,NaN,NaN,13.0,33.0,53.0,42.0,68.0,52.0,151.0
268,令嬢薔薇図鑑,2013-09-11,NaN,NaN,NaN,NaN,NaN,NaN,6.0,35.0,57.0,77.0,25.0,44.0,117.0
269,ローズ家の双子達,2013-09-11,NaN,NaN,NaN,NaN,NaN,NaN,3.0,12.0,47.0,45.0,26.0,78.0,106.0
317,昭和B級下手喰い道,2016-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,43.0,55.0,137.0
318,女化生舞楽図,2016-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,47.0,63.0,141.0


### average top

In [17]:
average_top = ura.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('average_exclude_debut <= 100') \
                 [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

average_top

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
185,神風,1,39.400000,2008-08-27
261,真夏の憂愁夫人,2,44.857143,2012-07-18
269,ローズ家の双子達,3,52.333333,2013-09-11
268,令嬢薔薇図鑑,4,59.166667,2013-09-11
130,昭和恋々幻燈館,5,62.692308,2005-06-22
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,6,63.222222,2009-08-26
195,鬼帝の剣,7,64.300000,2008-11-19
264,快恠奇奇,8,66.500000,2013-02-20
181,コトダマ,9,67.000000,2008-01-23


In [37]:
with open('../data_ura/average_top.md', 'w', encoding='utf8') as file:
    file.write('\#|曲名|平均\n')
    file.write('---:|---|---:\n')
    for no in average_top.index:
        file.write(f'{average_top.rank_average_exclude_debut.at[no]}|{average_top.title.at[no]}|{average_top.average_exclude_debut.at[no]:.2f}\n')

In [19]:
highest_top = ura.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('highest_exclude_debut <= 50') \
                 [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'highest', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int, 'highest': int})

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,highest,date
no,,,,,
261,真夏の憂愁夫人,1,9,9,2012-07-18
274,黒百合隠密カゲキダン,2,10,4,2013-09-11
269,ローズ家の双子達,3,12,3,2013-09-11
185,神風,4,14,13,2008-08-27
215,堕天國宣戦,4,14,7,2009-10-21
206,お毒味LADY,6,18,13,2009-08-26
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,7,19,17,2009-08-26
195,鬼帝の剣,8,23,5,2008-11-19
268,令嬢薔薇図鑑,9,25,6,2013-09-11


In [20]:
tmp = ura.loc[:, :]
tmp['over100_mean'] = tmp.over100 / tmp.count_all
tmp.sort_values(['over100_mean', 'date'], ascending=[False, True]) \
   .query('over100_mean >= 0.9') \
   [['title', 'over100', 'over100_mean', 'date']]

,title,over100,over100_mean,date
no,,,,
180,Nocturnal,12,1.000000,2007-12-12
192,薨,11,1.000000,2008-08-27
193,小さき者への贖罪のためのアリア,11,1.000000,2008-08-27
203,Hell's Maria,10,1.000000,2009-08-19
211,世紀末ゲネシス,10,1.000000,2009-08-26
216,寶島,10,1.000000,2009-10-21
218,恋の翼は真珠色の骨を持ち,10,1.000000,2010-03-17
223,Adieu,10,1.000000,2010-03-17
227,灰桜,9,1.000000,2010-08-25


In [21]:
# 未投票
zero = ['日曜日のシエスタ', 'La vie en rose', 'Féliciter', 'Nocturnal', '寶島', '逢魔蛾城の伯爵', '小夜すみれ ～ある夜の森の小さなお話', '魂ノ代']
ura.query('title in @zero')

,title,date,initial,kind,r2019u,r2007,r2008,r2009,r2010,r2011,...,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate,over100_mean
no,,,,,,,,,,,,,,,,,,,,,
55,日曜日のシエスタ,1997-12-17,r2005,"[SA, ST]",159.0,129.0,208.0,179.0,171.0,169.0,...,16,13,156.0,146.0,165.0,148.0,3,0.1875,108.0,0.8125
91,La vie en rose,2001-07-25,r2005,[OA],174.0,187.0,196.0,221.0,155.0,211.0,...,16,13,197.0,192.0,191.0,178.0,3,0.1875,108.0,0.8125
106,Féliciter,2003-04-23,r2005,[SA],181.0,168.0,174.0,215.0,179.0,223.0,...,16,13,180.0,170.0,201.0,188.0,3,0.1875,108.0,0.8125
180,Nocturnal,2007-12-12,r2008,[SA],194.0,NaN,154.0,202.0,192.0,223.0,...,12,12,205.0,204.0,188.0,204.0,0,0.0000,180.0,1.0000
216,寶島,2009-10-21,r2010,"[S-2, CA]",191.0,NaN,NaN,NaN,109.0,147.0,...,10,10,188.0,198.0,144.0,170.0,0,0.0000,180.0,1.0000
247,逢魔蛾城の伯爵,2011-06-29,r2011,[SA],181.0,NaN,NaN,NaN,NaN,123.0,...,9,9,161.0,161.0,162.0,165.0,0,0.0000,180.0,1.0000
296,小夜すみれ ～ある夜の森の小さなお話,2015-01-21,r2015,[SA],194.0,NaN,NaN,NaN,NaN,NaN,...,5,5,192.0,191.0,187.0,174.0,0,0.0000,180.0,1.0000
311,魂ノ代,2015-10-21,r2015,[S-2],194.0,NaN,NaN,NaN,NaN,NaN,...,5,5,211.0,210.0,204.0,210.0,0,0.0000,180.0,1.0000


In [22]:
ura[ura.kind.apply(lambda k: 'S-1' in k)]

,title,date,initial,kind,r2019u,r2007,r2008,r2009,r2010,r2011,...,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate,over100_mean
no,,,,,,,,,,,,,,,,,,,,,
12,フラワーチャイルド,1988-07-25,r2005,"[OA, S-1, CA]",165.0,187.0,197.0,221.0,214.0,223.0,...,16,13,183.0,175.0,172.0,153.0,3,0.187500,108.0,0.812500
13,恋せよ乙女 -Love story of ZIPANG-,1992-07-07,r2005,"[OA, S-1, SA, CA]",NaN,36.0,48.0,58.0,74.0,87.0,...,16,13,26.0,19.0,14.0,19.0,12,0.750000,11.0,0.250000
43,Wish,1996-12-24,r2005,"[S-1, OT, ST, CA]",27.0,53.0,71.0,81.0,72.0,86.0,...,16,13,52.0,42.0,67.0,33.0,9,0.562500,35.0,0.437500
57,Anniversary of Angel,1997-12-17,r2005,"[S-1, OT, CA]",83.0,54.0,99.0,97.0,98.0,147.0,...,16,13,78.0,66.0,75.0,41.0,8,0.500000,39.0,0.500000
59,LABYRINTH,1998-10-21,r2005,"[OA, S-1, ST, CA]",NaN,59.0,142.0,83.0,171.0,111.0,...,16,13,56.0,45.0,81.0,47.0,7,0.437500,53.0,0.562500
120,pastel pure,2004-08-25,r2005,"[S-1, ST, CA]",167.0,104.0,160.0,184.0,179.0,211.0,...,16,13,172.0,160.0,139.0,116.0,3,0.187500,108.0,0.812500
181,コトダマ,2008-01-23,r2008,"[S-1, ST, CA]",NaN,NaN,8.0,32.0,35.0,38.0,...,12,12,10.0,9.0,8.0,14.0,11,0.916667,2.0,0.083333
195,鬼帝の剣,2008-11-19,r2009,"[S-1, ST, CA]",NaN,NaN,NaN,5.0,32.0,30.0,...,11,11,7.0,7.0,3.0,8.0,9,0.818182,8.0,0.181818
198,裸々イヴ新世紀,2009-01-21,r2009,"[S-1, OT, ST, CA]",6.0,NaN,NaN,9.0,29.0,45.0,...,11,11,28.0,24.0,10.0,12.0,7,0.636364,25.0,0.363636


In [23]:
my_before = ['桜の花は狂い咲き', '乙女の祈り', 'Rose Moon', '少女貴族', '熱帯性植物園', '遊月恋歌', '少女殉血', 'Animals on the Earth', '四神獣飼殺し', 'ALICE同罪イノセント', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
ura.query('title in @my_before')

,title,date,initial,kind,r2019u,r2007,r2008,r2009,r2010,r2011,...,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate,over100_mean
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,r2005,"[SA, OA, OT]",64.0,86.0,126.0,116.0,109.0,96.0,...,16,13,58.0,46.0,97.0,64.0,7,0.437500,53.0,0.562500
209,Animals on the Earth,2009-08-26,r2010,[OA],50.0,NaN,NaN,NaN,35.0,45.0,...,10,10,49.0,49.0,48.0,27.0,5,0.500000,39.0,0.500000
234,四神獣飼殺し,2010-09-26,r2011,[OA],132.0,NaN,NaN,NaN,NaN,27.0,...,9,9,61.0,63.0,38.0,74.0,4,0.444444,50.0,0.555556
299,陸と海と空と,2015-09-09,r2015,"[OA, CA]",53.0,NaN,NaN,NaN,NaN,NaN,...,5,5,15.0,17.0,26.0,43.0,3,0.600000,29.0,0.400000
309,緋ノ糸輪廻ノGEMINI,2015-09-09,r2015,[OA],122.0,NaN,NaN,NaN,NaN,NaN,...,5,5,40.0,47.0,47.0,85.0,3,0.600000,29.0,0.400000


In [24]:
my = ['桜の花は狂い咲き', "Nous Deux C'est Pour La Vie", '月夜のピエレット', '神風', 'Animals on the Earth', '絶國TEMPEST', '四神獣飼殺し', 'La vérité', '真夏の憂愁夫人', '令嬢薔薇図鑑', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
ura.query('title in @my')

,title,date,initial,kind,r2019u,r2007,r2008,r2009,r2010,r2011,...,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100,under100_rate,rank_under100_rate,over100_mean
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,r2005,"[SA, OA, OT]",64.0,86.0,126.0,116.0,109.0,96.0,...,16,13,58.0,46.0,97.0,64.0,7,0.437500,53.0,0.562500
37,Nous Deux C'est Pour La Vie,1995-10-18,r2005,"[S-2, CA]",75.0,44.0,65.0,80.0,118.0,117.0,...,16,13,60.0,48.0,20.0,26.0,7,0.437500,53.0,0.562500
49,月夜のピエレット,1997-05-21,r2005,"[OA, OT, S-2, ST, CA]",82.0,94.0,172.0,107.0,104.0,92.0,...,16,13,81.0,68.0,97.0,64.0,6,0.375000,67.0,0.625000
185,神風,2008-08-27,r2009,"[OA, CA]",NaN,NaN,NaN,13.0,21.0,14.0,...,11,11,1.0,1.0,14.0,4.0,10,0.909091,3.0,0.090909
209,Animals on the Earth,2009-08-26,r2010,[OA],50.0,NaN,NaN,NaN,35.0,45.0,...,10,10,49.0,49.0,48.0,27.0,5,0.500000,39.0,0.500000
228,絶國TEMPEST,2010-09-26,r2011,"[OA, CA]",106.0,NaN,NaN,NaN,NaN,53.0,...,9,9,117.0,123.0,73.0,70.0,3,0.333333,75.0,0.666667
234,四神獣飼殺し,2010-09-26,r2011,[OA],132.0,NaN,NaN,NaN,NaN,27.0,...,9,9,61.0,63.0,38.0,74.0,4,0.444444,50.0,0.555556
258,La vérité,2012-07-18,r2012,[OA],NaN,NaN,NaN,NaN,NaN,NaN,...,8,8,27.0,27.0,19.0,48.0,5,0.625000,27.0,0.375000
261,真夏の憂愁夫人,2012-07-18,r2012,"[OA, CA]",NaN,NaN,NaN,NaN,NaN,NaN,...,8,8,2.0,2.0,10.0,1.0,7,0.875000,4.0,0.125000
